# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Files/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
import numpy as np

In [0]:
train_body = pd.read_csv(project_path+'train_bodies.csv')

In [0]:
train_stance = pd.read_csv(project_path+'train_stances.csv')

In [0]:
dataset = pd.merge(train_body,train_stance, on='Body ID')

In [301]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [302]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [304]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer_article = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer_headline = Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
tokenizer_article.fit_on_texts(dataset['articleBody'])

In [0]:
tokenizer_headline.fit_on_texts(dataset['Headline'])

In [317]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [318]:
print(tokenizer_article.word_index)
print(tokenizer_article.word_counts)
print(tokenizer_article.word_docs)
print(tokenizer_article.document_count)

{'the': 1, 'to': 2, 'a': 3, 'of': 4, 'in': 5, 'and': 6, 'that': 7, 'is': 8, 'was': 9, 'on': 10, 'for': 11, 'said': 12, 'he': 13, 'with': 14, 'it': 15, 'his': 16, 'have': 17, 'as': 18, 'by': 19, 'has': 20, 'from': 21, 'at': 22, 'be': 23, 'an': 24, 'not': 25, 'are': 26, 'been': 27, '”': 28, 'but': 29, 'this': 30, 'had': 31, 'who': 32, 'they': 33, 'after': 34, 'i': 35, 'were': 36, 'we': 37, 'will': 38, 'about': 39, 'one': 40, 'or': 41, 'which': 42, 'she': 43, 'video': 44, 'apple': 45, 'up': 46, 'would': 47, 'her': 48, 'state': 49, 'their': 50, 'also': 51, 'more': 52, 'when': 53, 'told': 54, 'out': 55, 'isis': 56, 'all': 57, 'no': 58, 'new': 59, 'people': 60, 'there': 61, 'you': 62, 'its': 63, 'if': 64, 'him': 65, 'news': 66, 'what': 67, 'could': 68, 'man': 69, 'year': 70, 'islamic': 71, 'time': 72, 'some': 73, 'al': 74, 'according': 75, 'watch': 76, 'over': 77, 'group': 78, 'into': 79, 'so': 80, 'first': 81, 'last': 82, 'being': 83, 'just': 84, 'u': 85, 's': 86, '—': 87, 'government': 88,

In [319]:
print(tokenizer_headline.word_index)
print(tokenizer_headline.word_counts)
print(tokenizer_headline.word_docs)
print(tokenizer_headline.document_count)

{'in': 1, 'to': 2, 'of': 3, 'isis': 4, 'the': 5, 'a': 6, 'is': 7, 'on': 8, 'for': 9, 'and': 10, 'by': 11, 'at': 12, 'after': 13, 'with': 14, 'foley': 15, 'james': 16, 'video': 17, 'journalist': 18, 'michael': 19, 'as': 20, 'says': 21, 'islamic': 22, 'state': 23, 'boko': 24, 'not': 25, 'haram': 26, 'was': 27, 'brown': 28, 'apple': 29, 'have': 30, 'claims': 31, 'audio': 32, 'american': 33, 'kim': 34, 'ebola': 35, 'his': 36, 'has': 37, 'us': 38, 'jong': 39, 'who': 40, 'u': 41, 's': 42, 'man': 43, 'from': 44, 'watch': 45, 'report': 46, 'new': 47, 'it': 48, 'reportedly': 49, 'missing': 50, 'steve': 51, 'reports': 52, 'shooting': 53, 'militants': 54, 'kidnapped': 55, 'that': 56, 'leader': 57, 'jobs': 58, 'killed': 59, 'wright': 60, 'dead': 61, 'fighters': 62, 'are': 63, 'be': 64, 'woman': 65, 'hoax': 66, 'shot': 67, 'girls': 68, 'meteorite': 69, 'alleged': 70, 'been': 71, 'up': 72, 'off': 73, 'ceasefire': 74, 'he': 75, 'beheading': 76, 'christian': 77, 'beheads': 78, 'no': 79, 'un': 80, 'cla

In [0]:
texts = dataset['articleBody']

In [321]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [322]:
texts.dtype

dtype('O')

In [0]:
articles = []
articles = texts.apply(sent_tokenize)

In [324]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

## Check 2:

first element of texts and articles should be as given below. 

In [325]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [326]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

In [0]:
data = np.zeros((len(articles),MAX_SENTS, MAX_SENT_LENGTH))

In [329]:
print(data.shape)
print(articles.shape)

(49972, 20, 20)
(49972,)


In [0]:
for i in range(0, len(articles)):
  articlelist = []
  j=0
  for j in range(len(articles[i])):
    sentenceencoded = [tokenizer_article.word_index[word] for word in text_to_word_sequence(articles[i][j])][0:20]
    sentencepadded = np.pad(np.array(sentenceencoded), mode='constant', constant_values=(0), pad_width=(0, 20 - len(sentenceencoded)))
    articlelist.append(list(sentencepadded))
   
  while len(articlelist)<20:
    seq = [0 for n in range(0,20)]
    articlelist.append(seq)
  
  #print(len(articlelist))  
  data[i,:20,:20]= np.array(articlelist[:20])

### Check 3:

Accessing first element in data should give something like given below.

In [333]:
data[0, :, :]

array([[3.0000e+00, 5.0000e+02, 5.3200e+02, 7.1170e+03, 7.9000e+01,
        3.0000e+00, 3.6640e+03, 3.1800e+02, 5.0000e+00, 4.5760e+03,
        3.6900e+02, 4.0000e+00, 1.6680e+03, 2.8860e+03, 1.0000e+00,
        8.8000e+01, 1.2000e+01, 4.4400e+02, 0.0000e+00, 0.0000e+00],
       [7.2700e+02, 9.4000e+01, 1.0320e+03, 3.0000e+00, 2.9630e+03,
        1.7780e+03, 7.0000e+00, 1.7100e+02, 3.0000e+00, 1.2230e+03,
        1.0950e+03, 2.0620e+03, 7.7900e+02, 1.5400e+02, 1.0000e+00,
        2.9590e+03, 4.7100e+02, 1.0000e+00, 5.3300e+02, 2.2700e+02],
       [8.8000e+01, 1.0330e+03, 4.0270e+03, 2.2750e+03, 1.2000e+01,
        3.0000e+00, 1.0520e+03, 3.2200e+03, 1.9000e+01, 1.0000e+00,
        8.8000e+01, 2.0000e+00, 1.7250e+03, 1.0000e+00, 5.1400e+02,
        1.9560e+03, 1.5000e+01, 9.0000e+00, 3.0000e+00, 3.0350e+03],
       [1.7400e+02, 3.7730e+03, 9.8000e+02, 1.9100e+02, 2.4870e+03,
        4.2000e+01, 6.6890e+03, 1.6650e+03, 1.2100e+03, 5.0000e+00,
        1.3146e+04, 1.7648e+04, 1.0000e+00, 7

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = []
articles_heading = []

from nltk.tokenize import sent_tokenize
texts_heading = list(dataset.Headline)
articles_heading = [sent_tokenize(text) for text in texts_heading ]

In [0]:
data_heading = np.zeros((len(articles_heading), 5, MAX_SENT_LENGTH))

for i in range(0, len(articles_heading)):
  article_headinglist = []
  j=0
  for j in range(len(articles_heading[i])):
    h_sentenceencoded = [tokenizer_headline.word_index[word] for word in text_to_word_sequence(articles_heading[i][j])][0:20]
    h_sentencepadded = np.pad(np.array(h_sentenceencoded), mode='constant', constant_values=(0), pad_width=(0, 20 - len(h_sentenceencoded)))
    article_headinglist.append(list(h_sentencepadded))  
  while len(article_headinglist)<5:
    seq = [0 for n in range(0,20)]
    article_headinglist.append(seq)
  
  #print(len(articlelist))  
  if len(article_headinglist) <= 5:
    headingdata[i,:5,:20]= np.array(article_headinglist[:5])
  else: 
    break
    headingdata[i,:5,:20]= np.array(article_headinglist[:5])   
 

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(dataset.Stance)

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [337]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels.index[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
split_index = int((1 - VALIDATION_SPLIT) * len(data))

x_train = data[:split_index]

x_val = data[split_index:]

split_index_lbl = int((1 - VALIDATION_SPLIT) * len(labels))

y_train = labels[:split_index_lbl]

y_val = labels[split_index_lbl:]

split_index_heading = int((1 - VALIDATION_SPLIT) * len(data))

x_heading_train = data_heading[:split_index_heading ]
 
x_heading_val = data_heading[split_index_heading :]

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [341]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977,)
(9995, 20, 20)
(9995,)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [0]:
tokenizer_article.fit_on_texts(dataset.articleBody)
word_index=tokenizer_article.word_index

In [0]:
vocab_size = len(tokenizer_article.word_counts)

In [362]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))

Loaded 400000 word vectors.


In [363]:
embedding_matrix.shape

(27427, 100)

In [364]:
print(vocab_size)

27427


In [365]:
tokenizer.word_index.items()

dict_items([('<unw>', 1), ('the', 2), ('to', 3), ('a', 4), ('of', 5), ('in', 6), ('and', 7), ('that', 8), ('is', 9), ('was', 10), ('on', 11), ('for', 12), ('with', 13), ('he', 14), ('his', 15), ('have', 16), ('it', 17), ('by', 18), ('as', 19), ('has', 20), ('from', 21), ('be', 22), ('said', 23), ('at', 24), ('an', 25), ('not', 26), ('been', 27), ('are', 28), ('had', 29), ('who', 30), ('but', 31), ('this', 32), ('they', 33), ('were', 34), ('after', 35), ('will', 36), ('about', 37), ('or', 38), ('which', 39), ('we', 40), ('i', 41), ('one', 42), ('would', 43), ('their', 44), ('told', 45), ('she', 46), ('also', 47), ('apple', 48), ('her', 49), ('when', 50), ('up', 51), ('new', 52), ('more', 53), ('its', 54), ('out', 55), ('all', 56), ('state', 57), ('no', 58), ('could', 59), ('what', 60), ('video', 61), ('people', 62), ('you', 63), ('islamic', 64), ('if', 65), ('some', 66), ('into', 67), ('over', 68), ('there', 69), ('just', 70), ('man', 71), ('being', 72), ('isis', 73), ('news', 74), ('fi

In [0]:
for word, i in tokenizer_article.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
import re
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score

### Model

In [0]:
sequence_length =20
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(dataset['articleBody'].values)

In [0]:
X = tokenizer.texts_to_sequences(dataset['articleBody'].values)

In [0]:
X = pad_sequences(X, sequence_length)

In [0]:
le = LabelEncoder()
y=le.fit_transform(list(dataset['Stance'].values))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [375]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(34980, 20)
(14992, 20)
(34980,)
(14992,)


### Compile and fit the model

In [0]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,5000,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=20,
                    trainable=True))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(32)))
model.add(Dropout(0.25))
model.add(Dense(units=4, activation='softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [377]:
batch_size = 128

final_model = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, verbose=1, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 100000000 elements. This may consume a large amount of memory.
  num_elements)


Train on 27984 samples, validate on 6996 samples
Epoch 1/10


ResourceExhaustedError: ignored

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)